# 🚀 ChemLLM Training - Interactive HuggingFace Integration

This notebook provides an interactive environment for training ChemLLM using HuggingFace Transformers. It's based on the `simple_training.py` script but allows for experimentation and parameter tuning.

## 🎯 Key Features
- **90% Code Reduction**: From 500+ lines to ~100 lines using HuggingFace
- **Interactive Experimentation**: Modify parameters and see results immediately
- **Memory Efficient**: Built-in optimizations and gradient checkpointing
- **Professional Training**: HF Trainer with automatic mixed precision

## 📋 What You'll Learn
1. How to replace custom training loops with HuggingFace Trainer
2. Efficient data loading using HF Datasets
3. Model optimization techniques (Flash Attention, gradient checkpointing)
4. Interactive hyperparameter tuning

## 📦 Import Required Libraries

First, let's import all the necessary libraries for training our ChemLLM model.

In [1]:
import logging
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM, 
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from datasets import load_dataset
import warnings

# Suppress TensorFlow warnings for cleaner output
warnings.filterwarnings("ignore")

print("✅ All libraries imported successfully!")
print(f"🔥 PyTorch version: {torch.__version__}")
print(f"🤗 Transformers available")
print(f"🚀 CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"📊 GPU: {torch.cuda.get_device_name()}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

/home/alibina/repo/Training_LLM/chemLLM/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-05 18:35:18.395024: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-05 18:35:18.420370: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751733318.440813  291567 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751733318.446341  291567 cud

✅ All libraries imported successfully!
🔥 PyTorch version: 2.7.1+cu126
🤗 Transformers available
🚀 CUDA available: True
📊 GPU: NVIDIA GeForce RTX 4050 Laptop GPU
💾 GPU Memory: 6.0 GB
📊 GPU: NVIDIA GeForce RTX 4050 Laptop GPU
💾 GPU Memory: 6.0 GB


## ⚙️ Setup Logging and GPU Memory Management

Configure logging for detailed training progress and clear GPU memory for optimal performance.

In [2]:
# Setup logging for detailed output
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Clear GPU memory cache for optimal training
torch.cuda.empty_cache()

# Global variables for experiment tracking
experiment_results = []

print("🔧 Logging configured")
print("🧹 GPU memory cleared")
print("📊 Ready for training experiments!")

🔧 Logging configured
🧹 GPU memory cleared
📊 Ready for training experiments!


## 🎛️ Create Training Configuration

Define the training configuration function. This replaces 100+ lines of custom config with HuggingFace's `TrainingArguments`.

In [3]:
def create_simple_training_config(
    output_dir: str = "./results",
    num_epochs: int = 1,
    batch_size: int = 4,
    learning_rate: float = 3e-4,
    max_length: int = 256,
    **kwargs
) -> TrainingArguments:
    """
    Create training configuration with HuggingFace TrainingArguments.
    
    This replaces 100+ lines of custom training configuration with
    a simple, well-tested configuration system.
    """
    return TrainingArguments(
        output_dir=output_dir,
        
        # Training setup
        num_train_epochs=num_epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        learning_rate=learning_rate,
        
        # Optimization (built-in)
        bf16=torch.cuda.is_bf16_supported(),  # Use BF16 if supported, otherwise FP32
        fp16=False,  # Disable FP16 to avoid gradient scaling issues
        gradient_accumulation_steps=4,  # Memory efficiency
        dataloader_num_workers=4,  # Parallel data loading
        
        # Evaluation and checkpointing (built-in)
        eval_strategy="steps",
        eval_steps=100,
        save_steps=500,
        save_total_limit=3,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        
        # Logging (built-in)
        logging_steps=10,
        logging_dir=f"{output_dir}/logs",
        report_to="none",  # Set to "wandb" for experiment tracking
        
        # Performance
        dataloader_pin_memory=True,
        dataloader_persistent_workers=True,
        
        **kwargs
    )

print("✅ Training configuration function defined!")
print("🎯 Features: BF16/FP32 precision, gradient accumulation, automatic checkpointing")

✅ Training configuration function defined!
🎯 Features: Mixed precision, gradient accumulation, automatic checkpointing


## 📊 Load and Prepare Dataset

Load the ChemPile dataset using HuggingFace Datasets. This replaces ~200 lines of custom data loading code.

In [4]:
def load_and_prepare_data(
    model_name: str = "gpt2",
    dataset_name: str = "iAli61/chempile-education-dedup",
    max_length: int = 256,
    max_samples: int = 1000,  # Small for demo
    test_split_size: float = 0.1
):
    """
    Load and prepare data using HuggingFace Datasets.
    
    This replaces ~200 lines of custom data loading with
    HuggingFace's optimized data pipeline.
    """
    print(f"🔤 Loading tokenizer: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    # Set pad token
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    print(f"📚 Loading dataset: {dataset_name}")
    # Load dataset
    try:
        dataset = load_dataset(dataset_name, split="train")
        print(f"📈 Original dataset size: {len(dataset):,}")
        
        # Limit size for demo
        if max_samples and len(dataset) > max_samples:
            dataset = dataset.select(range(max_samples))
            print(f"🎯 Limited to {max_samples:,} samples for demo")
            
    except Exception as e:
        print(f"❌ Failed to load dataset {dataset_name}: {e}")
        print("🔄 Falling back to synthetic dataset for demo")
        
        # Create a small synthetic dataset for demo
        synthetic_data = [
            "This is a chemical compound with molecular formula C6H12O6.",
            "The reaction produces water and carbon dioxide as byproducts.",
            "Catalysts are substances that increase the rate of chemical reactions.",
            "Organic chemistry deals with carbon-containing compounds.",
            "The periodic table organizes elements by their atomic properties."
        ] * (max_samples // 5)
        
        from datasets import Dataset
        dataset = Dataset.from_dict({"text": synthetic_data})
    
    # Tokenization function
    def tokenize_function(examples):
        # Tokenize text with proper padding and truncation
        tokens = tokenizer(
            examples["text"],
            truncation=True,
            padding="max_length",  # Pad to max_length for consistent sizes
            max_length=max_length,
            return_tensors="pt"
        )
        
        return {
            "input_ids": tokens["input_ids"],
            "attention_mask": tokens["attention_mask"]
        }
    
    # Apply tokenization
    print("🔧 Tokenizing dataset...")
    tokenized_dataset = dataset.map(
        tokenize_function,
        batched=True,
        remove_columns=dataset.column_names,
        desc="Tokenizing"
    )
    
    # Add labels for causal language modeling
    def add_labels(examples):
        # For causal language modeling, labels should be the same as input_ids
        # The DataCollatorForLanguageModeling will handle the shifting during training
        examples["labels"] = examples["input_ids"].copy()
        return examples
    
    tokenized_dataset = tokenized_dataset.map(add_labels, batched=True)
    
    # Create train/validation split
    split_dataset = tokenized_dataset.train_test_split(
        test_size=test_split_size,
        seed=42
    )
    
    print(f"📊 Dataset splits: train={len(split_dataset['train']):,}, val={len(split_dataset['test']):,}")
    print(f"🔍 Sample tokens: {split_dataset['train'][0]['input_ids'][:10]}...")

    return split_dataset, tokenizer

print("✅ Data loading function defined!")
print("🎯 Features: HF Datasets, automatic tokenization, train/val split")

✅ Data loading function defined!
🎯 Features: HF Datasets, automatic tokenization, train/val split


## 🤖 Create and Configure Model

Load the GPT-2 model with optimizations. This replaces ~150 lines of custom model setup.

In [5]:
def create_model(model_name: str = "gpt2", use_flash_attention: bool = False):
    """
    Create model with HuggingFace integration.
    
    This replaces ~150 lines of custom model setup with
    HuggingFace's optimized model loading.
    """
    print(f"🤖 Loading model: {model_name}")
    
    model_kwargs = {
        "torch_dtype": torch.float16,  # Mixed precision
    }
    
    # Enable Flash Attention if requested and available
    if use_flash_attention:
        try:
            model_kwargs["attn_implementation"] = "flash_attention_2"
            print("⚡ Enabled Flash Attention 2")
        except Exception as e:
            print(f"⚠️ Flash Attention not available: {e}")
    
    model = AutoModelForCausalLM.from_pretrained(model_name, **model_kwargs)
    
    # Enable gradient checkpointing for memory efficiency
    model.gradient_checkpointing_enable()
    
    print(f"✅ Model loaded with {model.num_parameters():,} parameters")
    print(f"💾 Memory efficient gradient checkpointing enabled")
    
    return model

print("✅ Model creation function defined!")
print("🎯 Features: Mixed precision, gradient checkpointing, Flash Attention support")

✅ Model creation function defined!
🎯 Features: Mixed precision, gradient checkpointing, Flash Attention support


## 🧪 Quick Experiment - Load Data and Model

Let's run a quick experiment to load some data and create a model. You can modify the parameters below:

In [6]:
# 🎛️ Experiment Parameters - Modify these to experiment!
MODEL_NAME = "gpt2"  # Try: "gpt2", "gpt2-medium", "distilgpt2"
DATASET_NAME = "iAli61/chempile-education-dedup"
MAX_SAMPLES = 50  # Start small for quick experiments
MAX_LENGTH = 128  # Shorter for faster processing

# Load data
print("🚀 Starting quick experiment...")
dataset, tokenizer = load_and_prepare_data(
    model_name=MODEL_NAME,
    dataset_name=DATASET_NAME,
    max_samples=MAX_SAMPLES,
    max_length=MAX_LENGTH
)

# Create model
model = create_model(MODEL_NAME, use_flash_attention=False)

print("\n🎉 Quick experiment complete!")
print(f"📊 Loaded {len(dataset['train'])} training samples")
print(f"🔤 Vocabulary size: {tokenizer.vocab_size:,}")
print(f"🤖 Model parameters: {model.num_parameters():,}")

# Save for later use
current_dataset = dataset
current_tokenizer = tokenizer  
current_model = model

🚀 Starting quick experiment...
🔤 Loading tokenizer: gpt2
📚 Loading dataset: iAli61/chempile-education-dedup
❌ Failed to load dataset iAli61/chempile-education-dedup: Dataset 'iAli61/chempile-education-dedup' doesn't exist on the Hub or cannot be accessed.
🔄 Falling back to synthetic dataset for demo
🔧 Tokenizing dataset...
📚 Loading dataset: iAli61/chempile-education-dedup
❌ Failed to load dataset iAli61/chempile-education-dedup: Dataset 'iAli61/chempile-education-dedup' doesn't exist on the Hub or cannot be accessed.
🔄 Falling back to synthetic dataset for demo
🔧 Tokenizing dataset...


Map: 100%|██████████| 50/50 [00:00<00:00, 7095.04 examples/s]



📊 Dataset splits: train=45, val=5
🔍 Sample tokens: [21979, 3400, 6448, 389, 15938, 326, 2620, 262, 2494, 286]...
🤖 Loading model: gpt2
✅ Model loaded with 124,439,808 parameters
💾 Memory efficient gradient checkpointing enabled

🎉 Quick experiment complete!
📊 Loaded 45 training samples
🔤 Vocabulary size: 50,257
🤖 Model parameters: 124,439,808
✅ Model loaded with 124,439,808 parameters
💾 Memory efficient gradient checkpointing enabled

🎉 Quick experiment complete!
📊 Loaded 45 training samples
🔤 Vocabulary size: 50,257
🤖 Model parameters: 124,439,808


## 🔄 Setup Data Collator

Configure the data collator for causal language modeling. This handles batching and label shifting automatically.

In [7]:
# Create data collator for causal language modeling
# This automatically handles:
# - Batching sequences
# - Padding to the same length  
# - Shifting labels for next-token prediction

data_collator = DataCollatorForLanguageModeling(
    tokenizer=current_tokenizer,
    mlm=False,  # False = Causal LM (next token prediction)
    return_tensors="pt"
)

print("✅ Data collator configured for causal language modeling")
print("🎯 Features: Automatic batching, padding, and label shifting")

# Test the data collator
sample_data = [current_dataset['train'][0], current_dataset['train'][1]]
batch = data_collator(sample_data)

print(f"\n🔍 Batch inspection:")
print(f"  Input shape: {batch['input_ids'].shape}")
print(f"  Labels shape: {batch['labels'].shape}")
print(f"  Input sample: {batch['input_ids'][0][:10].tolist()}")
print(f"  Label sample: {batch['labels'][0][:10].tolist()}")

# The labels are shifted by the data collator for next-token prediction
input_first_10 = batch['input_ids'][0][:10].tolist()
label_first_10 = batch['labels'][0][:10].tolist()
print(f"  Labels shifted correctly: {input_first_10[1:] == label_first_10[:-1]}")

✅ Data collator configured for causal language modeling
🎯 Features: Automatic batching, padding, and label shifting

🔍 Batch inspection:
  Input shape: torch.Size([2, 13])
  Labels shape: torch.Size([2, 13])
  Input sample: [21979, 3400, 6448, 389, 15938, 326, 2620, 262, 2494, 286]
  Label sample: [21979, 3400, 6448, 389, 15938, 326, 2620, 262, 2494, 286]
  Labels shifted correctly: False


## 🎯 Initialize Trainer

Create the HuggingFace Trainer. This replaces 100+ lines of custom training loop code.

In [ ]:
# 🎛️ Training Configuration - Experiment with these parameters!
OUTPUT_DIR = "./notebook_results"
NUM_EPOCHS = 1  # Start with 1 for quick experiments
BATCH_SIZE = 2  # Small batch for demo (increase for real training)
LEARNING_RATE = 5e-4  # Try: 3e-4, 5e-4, 1e-3
EVAL_STEPS = 50  # Evaluate every N steps

# Create training configuration
training_args = create_simple_training_config(
    output_dir=OUTPUT_DIR,
    num_epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    eval_steps=EVAL_STEPS
)

# Create trainer
trainer = Trainer(
    model=current_model,
    args=training_args,
    train_dataset=current_dataset["train"],
    eval_dataset=current_dataset["test"],
    data_collator=data_collator,
    tokenizer=current_tokenizer,
)

print("✅ Trainer initialized successfully!")
print(f"🎯 Configuration:")
print(f"  • Epochs: {NUM_EPOCHS}")
print(f"  • Batch size: {BATCH_SIZE}")
print(f"  • Learning rate: {LEARNING_RATE}")
print(f"  • Eval steps: {EVAL_STEPS}")
print(f"  • Output dir: {OUTPUT_DIR}")
print(f"  • Mixed precision: {training_args.fp16}")
print(f"  • Gradient accumulation: {training_args.gradient_accumulation_steps}")

# Store trainer for later use
current_trainer = trainer

## 🚀 Train the Model

Execute the training loop. This is where the magic happens - all the complex training logic is handled by HuggingFace!

In [ ]:
import time

print("🚀 Starting training...")
print("💡 This replaces 100+ lines of custom training loop with a single line!")

start_time = time.time()

# Train the model - this is the entire training loop!
train_result = current_trainer.train()

end_time = time.time()
training_time = end_time - start_time

print(f"\n✅ Training completed!")
print(f"⏱️  Training time: {training_time:.1f} seconds")
print(f"📊 Final training loss: {train_result.training_loss:.4f}")
print(f"🚀 Samples per second: {train_result.metrics['train_samples_per_second']:.1f}")
print(f"💾 Model saved to: {OUTPUT_DIR}")

# Save training results for later analysis
training_results = {
    'training_loss': train_result.training_loss,
    'training_time': training_time,
    'samples_per_second': train_result.metrics['train_samples_per_second'],
    'total_steps': train_result.metrics['train_steps']
}

experiment_results.append({
    'experiment': f'Training_{len(experiment_results)+1}',
    'config': {
        'model': MODEL_NAME,
        'epochs': NUM_EPOCHS,
        'batch_size': BATCH_SIZE,
        'learning_rate': LEARNING_RATE,
        'max_samples': MAX_SAMPLES
    },
    'results': training_results
})

print(f"\n📈 Experiment {len(experiment_results)} saved to experiment_results")

## 📊 Evaluate Model Performance

Evaluate the trained model on the validation set to see how well it learned.

In [ ]:
print("📊 Evaluating model performance...")

# Evaluate the model
eval_result = current_trainer.evaluate()

print(f"\n📈 Evaluation Results:")
print(f"  • Validation loss: {eval_result['eval_loss']:.4f}")
print(f"  • Perplexity: {torch.exp(torch.tensor(eval_result['eval_loss'])):.2f}")
print(f"  • Eval runtime: {eval_result['eval_runtime']:.1f}s")
print(f"  • Samples per second: {eval_result['eval_samples_per_second']:.1f}")

# Add evaluation results to our experiment tracking
training_results['eval_loss'] = eval_result['eval_loss']
training_results['perplexity'] = float(torch.exp(torch.tensor(eval_result['eval_loss'])))

print(f"\n✅ Evaluation complete!")
print(f"💡 Lower loss and perplexity = better performance")

# Compare with baseline (if we have multiple experiments)
if len(experiment_results) > 1:
    print(f"\n📊 Comparison with previous experiments:")
    for i, exp in enumerate(experiment_results):
        results = exp['results']
        print(f"  Experiment {i+1}: Loss={results['training_loss']:.4f}, "
              f"Eval Loss={results.get('eval_loss', 'N/A')}")
else:
    print(f"\n🔍 Run more experiments to compare results!")

## 📝 Generate Sample Text

Let's test our trained model by generating chemical text! Try different prompts and parameters.

In [ ]:
def generate_text(prompt, max_new_tokens=50, temperature=0.7, num_sequences=1):
    """Generate text using the trained model."""
    print(f"🤖 Generating text for prompt: '{prompt}'")
    
    # Prepare input
    inputs = current_tokenizer(prompt, return_tensors="pt")
    
    # Move to GPU if available
    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}
        current_model.cuda()
    
    # Generate text
    current_model.eval()
    with torch.no_grad():
        outputs = current_model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            num_return_sequences=num_sequences,
            temperature=temperature,
            do_sample=True,
            pad_token_id=current_tokenizer.eos_token_id
        )
    
    # Decode and display results
    generated_texts = []
    for i, output in enumerate(outputs):
        text = current_tokenizer.decode(output, skip_special_tokens=True)
        generated_texts.append(text)
        print(f"\n📝 Generated text {i+1}:")
        print(f"   {text}")
    
    return generated_texts

# 🧪 Try different prompts - modify these!
test_prompts = [
    "The chemical compound",
    "This reaction produces",
    "The catalyst increases",
    "Organic chemistry involves",
    "The molecular formula"
]

print("🎯 Testing the model with chemical prompts:")
print("=" * 50)

generated_samples = []
for prompt in test_prompts:
    generated = generate_text(
        prompt=prompt, 
        max_new_tokens=30,  # Adjust for longer/shorter text
        temperature=0.7,    # Lower = more focused, Higher = more creative
        num_sequences=1
    )
    generated_samples.extend(generated)
    print("-" * 30)

print(f"\n✅ Generated {len(generated_samples)} text samples!")
print("💡 Try modifying the prompts, temperature, and max_new_tokens above!")

## 🔬 Interactive Experimentation Zone

This section is for you to experiment with different parameters and configurations. Try the examples below or create your own!

In [ ]:
# 🎨 Custom Text Generation - Your Turn!
# Modify these parameters to experiment:

YOUR_PROMPT = "The synthesis of"  # ← Change this to your prompt!
MAX_TOKENS = 40                   # ← Adjust length
TEMPERATURE = 0.8                 # ← Creativity (0.1-1.0)
NUM_SAMPLES = 3                   # ← How many variations

print(f"🎯 Custom generation with your parameters:")
print(f"   Prompt: '{YOUR_PROMPT}'")
print(f"   Max tokens: {MAX_TOKENS}")
print(f"   Temperature: {TEMPERATURE}")
print(f"   Samples: {NUM_SAMPLES}")

custom_generated = generate_text(
    prompt=YOUR_PROMPT,
    max_new_tokens=MAX_TOKENS,
    temperature=TEMPERATURE,
    num_sequences=NUM_SAMPLES
)

print(f"\n🎉 Generated {len(custom_generated)} custom samples!")
print("💡 Try different values above to see how they affect the output!")

In [ ]:
# 🧪 Hyperparameter Experiment - Train with Different Settings
# Try training a new model with different parameters:

def quick_experiment(learning_rate, batch_size, max_samples=30):
    """Run a quick training experiment with different parameters."""
    print(f"\n🚀 Experiment: LR={learning_rate}, Batch={batch_size}, Samples={max_samples}")
    
    # Load fresh data and model
    exp_dataset, exp_tokenizer = load_and_prepare_data(
        model_name="gpt2",
        max_samples=max_samples,
        max_length=128
    )
    
    exp_model = create_model("gpt2")
    
    # New training config
    exp_training_args = create_simple_training_config(
        output_dir=f"./exp_lr{learning_rate}_bs{batch_size}",
        num_epochs=1,
        batch_size=batch_size,
        learning_rate=learning_rate,
        eval_steps=20
    )
    
    # Create trainer
    exp_trainer = Trainer(
        model=exp_model,
        args=exp_training_args,
        train_dataset=exp_dataset["train"],
        eval_dataset=exp_dataset["test"],
        data_collator=DataCollatorForLanguageModeling(exp_tokenizer, mlm=False),
        tokenizer=exp_tokenizer,
    )
    
    # Train
    start_time = time.time()
    result = exp_trainer.train()
    training_time = time.time() - start_time
    
    # Evaluate
    eval_result = exp_trainer.evaluate()
    
    experiment_data = {
        'learning_rate': learning_rate,
        'batch_size': batch_size,
        'max_samples': max_samples,
        'train_loss': result.training_loss,
        'eval_loss': eval_result['eval_loss'],
        'training_time': training_time,
        'samples_per_second': result.metrics['train_samples_per_second']
    }
    
    print(f"✅ Results: Train Loss={result.training_loss:.4f}, "
          f"Eval Loss={eval_result['eval_loss']:.4f}, Time={training_time:.1f}s")
    
    return experiment_data

# Uncomment and run experiments (warning: this will take some time!)
# exp1 = quick_experiment(learning_rate=3e-4, batch_size=2)
# exp2 = quick_experiment(learning_rate=5e-4, batch_size=2)  
# exp3 = quick_experiment(learning_rate=3e-4, batch_size=4)

print("🔬 Hyperparameter experiment function ready!")
print("💡 Uncomment the experiment lines above to run comparative tests")
print("⚠️ Each experiment takes 1-2 minutes to complete")

In [ ]:
# 📊 Experiment Results Analysis
# View and compare all your experiments

def analyze_experiments():
    """Display a summary of all experiments run in this session."""
    if not experiment_results:
        print("❌ No experiments found. Run some training first!")
        return
    
    print("📈 Experiment Results Summary:")
    print("=" * 60)
    
    for i, exp in enumerate(experiment_results):
        config = exp['config']
        results = exp['results']
        
        print(f"\n🧪 Experiment {i+1}:")
        print(f"   Model: {config['model']}")
        print(f"   Epochs: {config['epochs']}, Batch: {config['batch_size']}, LR: {config['learning_rate']}")
        print(f"   Samples: {config['max_samples']}")
        print(f"   📊 Train Loss: {results['training_loss']:.4f}")
        print(f"   📊 Eval Loss: {results.get('eval_loss', 'N/A')}")
        print(f"   ⏱️  Time: {results['training_time']:.1f}s")
        print(f"   🚀 Speed: {results['samples_per_second']:.1f} samples/sec")
    
    # Find best experiment
    if len(experiment_results) > 1:
        best_exp = min(experiment_results, 
                      key=lambda x: x['results'].get('eval_loss', float('inf')))
        best_idx = experiment_results.index(best_exp) + 1
        print(f"\n🏆 Best experiment: #{best_idx} (lowest eval loss)")

# Run analysis
analyze_experiments()

print(f"\n📝 Total experiments run: {len(experiment_results)}")
print("💡 The more experiments you run, the better you'll understand the model!")

## 🎉 Summary and Next Steps

Congratulations! You've successfully used HuggingFace to train a ChemLLM model with minimal code.

### 🚀 What You Accomplished
- ✅ **90% Code Reduction**: Replaced 500+ lines with ~100 lines using HuggingFace
- ✅ **Professional Training**: Used industry-standard HF Trainer with built-in optimizations
- ✅ **Interactive Experimentation**: Tested different hyperparameters and prompts
- ✅ **Memory Efficiency**: Leveraged gradient checkpointing and mixed precision
- ✅ **Model Evaluation**: Assessed performance with proper validation metrics

### 📚 Key Learnings
1. **HuggingFace Integration**: How to replace custom training loops with HF Trainer
2. **Data Pipeline**: Efficient data loading with HF Datasets
3. **Model Optimization**: Built-in features like Flash Attention and gradient checkpointing
4. **Experiment Tracking**: How to systematically compare different configurations

### 🔄 Next Steps
- **Scale Up**: Try larger models (gpt2-medium, gpt2-large)
- **More Data**: Use the full ChemPile dataset (remove max_samples limit)
- **Advanced Features**: Experiment with Flash Attention, different optimizers
- **Hyperparameter Search**: Use HF's built-in hyperparameter search
- **Production Deployment**: Save and deploy your best model

### 💡 Experiment Ideas
- Compare different learning rates (1e-4, 3e-4, 5e-4, 1e-3)
- Test different batch sizes and gradient accumulation
- Try different models (distilgpt2 for speed, gpt2-medium for quality)
- Experiment with different temperature values for generation
- Use different chemical prompts to test domain adaptation

In [ ]:
# 🛠️ Utility Functions for Your Experiments

def save_model_and_tokenizer(model, tokenizer, save_path="./final_model"):
    """Save your trained model and tokenizer for later use."""
    print(f"💾 Saving model and tokenizer to {save_path}")
    model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)
    print("✅ Model and tokenizer saved successfully!")

def load_saved_model(model_path="./final_model"):
    """Load a previously saved model and tokenizer."""
    print(f"📂 Loading model from {model_path}")
    model = AutoModelForCausalLM.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    print("✅ Model and tokenizer loaded successfully!")
    return model, tokenizer

def compare_models(prompts=["The chemical compound", "This reaction"]):
    """Compare text generation before and after training."""
    print("🔍 Comparing pre-trained vs fine-tuned model:")
    
    # Load fresh pre-trained model
    original_model = AutoModelForCausalLM.from_pretrained("gpt2")
    
    for prompt in prompts:
        print(f"\n📝 Prompt: '{prompt}'")
        
        # Original model
        inputs = current_tokenizer(prompt, return_tensors="pt")
        with torch.no_grad():
            outputs_orig = original_model.generate(**inputs, max_new_tokens=20, temperature=0.7)
        text_orig = current_tokenizer.decode(outputs_orig[0], skip_special_tokens=True)
        
        # Fine-tuned model  
        with torch.no_grad():
            outputs_fine = current_model.generate(**inputs, max_new_tokens=20, temperature=0.7)
        text_fine = current_tokenizer.decode(outputs_fine[0], skip_special_tokens=True)
        
        print(f"   🤖 Original GPT-2: {text_orig}")
        print(f"   🧪 Fine-tuned:     {text_fine}")

# Ready to use!
print("🛠️ Utility functions loaded:")
print("   • save_model_and_tokenizer() - Save your trained model")
print("   • load_saved_model() - Load a saved model")  
print("   • compare_models() - Compare before/after training")
print("\n🎯 Example usage:")
print("   save_model_and_tokenizer(current_model, current_tokenizer)")
print("   compare_models(['The synthesis of', 'Chemical bonds'])")
print("\n🎉 Happy experimenting!")